In [83]:
import csv
import os
import pandas as pd
import numpy as np
import functools
def conjunction(*conditions):
    return functools.reduce(np.logical_and, conditions)

os.chdir('/Users/macbook/Desktop/Data-Program-Files/Group-Projects/Team-Folder/')

FHV201801 = pd.read_csv("fhv_tripdata_2018-01.csv", parse_dates=[0], low_memory=False)
Weather_Data = pd.read_csv('NYC_Weather.csv', parse_dates=[0], low_memory=False)
Zone_Data = pd.read_csv('fhv_zone_lookup.csv', parse_dates=[0], low_memory=False)

FHV201801 = FHV201801.fillna(0)

FHV201801['Pickup_Date'] = FHV201801.Pickup_DateTime.dt.date

FHV201801 = FHV201801.groupby('Pickup_Date').apply(pd.DataFrame.sample, n=10000).reset_index(drop=True) #Get 10,000 random rows from each date

is_not_0_p =  FHV201801['PUlocationID']!=0 #no location 
is_not_264_p =  FHV201801['PUlocationID']!=264 #unknown location
is_not_265_p =  FHV201801['PUlocationID']!=265 #unknown location

is_not_0_d =  FHV201801['DOlocationID']!=0 #no location 
is_not_264_d =  FHV201801['DOlocationID']!=264 #unknown location
is_not_265_d =  FHV201801['DOlocationID']!=265 #unknown location

FHV201801 = FHV201801[conjunction(is_not_0_p,is_not_264_p,is_not_265_p,is_not_0_d,is_not_264_d,is_not_265_d)] #clean data
FHV201801['Pickup_DateTime'] = pd.to_datetime(FHV201801['Pickup_DateTime'])
FHV201801['DropOff_datetime'] = pd.to_datetime(FHV201801['DropOff_datetime'])
FHV201801['PUlocationID'] = pd.to_numeric(FHV201801['PUlocationID'])
FHV201801['DOlocationID'] = pd.to_numeric(FHV201801['DOlocationID'])

FHV201801['Pickup_DateTime2'] = FHV201801.Pickup_DateTime.dt.round('H')

Weather_Data['dt_iso'] = pd.to_datetime(Weather_Data['dt_iso'])
Weather_Data = Weather_Data.fillna(0)

Zone_Data['LocationID'] = pd.to_numeric(Zone_Data['LocationID'])
Zone_Data = Zone_Data.fillna(0)

Combined_Data = pd.merge(FHV201801, Weather_Data, how='left', left_on=['Pickup_DateTime2'], right_on=['dt_iso'])

Combined_Data = pd.merge(Combined_Data, Zone_Data, how='left', left_on=['PUlocationID'], right_on=['LocationID'])

Combined_Data = pd.merge(Combined_Data, Zone_Data, how='left', left_on=['DOlocationID'], right_on=['LocationID'])

Combined_Data = Combined_Data[['Pickup_DateTime', 'DropOff_datetime', 'PUlocationID', 'DOlocationID', 'SR_Flag', 'city_name', 'temp', 'humidity', 'wind_speed', 'weather_id', 'weather_main', 'weather_description', 'Borough_x', 'Zone_x', 'Borough_y', 'Zone_y']]

Combined_Data_Fixed = Combined_Data.rename(columns={'Borough_x': 'Pickup_Location', 'Zone_x': 'Pickup_Neighborhood', 'Borough_y': 'Dropoff_Location', 'Zone_y': 'Dropoff_Neighborhood'})

Combined_Data_Fixed.to_csv("RideShare_Data_201801.csv", index=False)


,Pickup_DateTime,DropOff_datetime,PUlocationID,DOlocationID,SR_Flag,Dispatching_base_number,Dispatching_base_num,Pickup_Date,Pickup_DateTime2,dt,...,weather_description,weather_icon,LocationID_x,Borough_x,Zone_x,service_zone_x,LocationID_y,Borough_y,Zone_y,service_zone_y
0,2018-01-01 18:11:34,2018-01-01 18:16:21,72.0,72.0,0.0,B02836,0.0,2018-01-01,2018-01-01 18:00:00,1514829600,...,sky is clear,01d,72,Brooklyn,East Flatbush/Remsen Village,Boro Zone,72,Brooklyn,East Flatbush/Remsen Village,Boro Zone
1,2018-01-01 15:06:45,2018-01-01 15:19:27,260.0,82.0,1.0,B02510,0.0,2018-01-01,2018-01-01 15:00:00,1514818800,...,sky is clear,01d,260,Queens,Woodside,Boro Zone,82,Queens,Elmhurst,Boro Zone
2,2018-01-01 05:24:11,2018-01-01 05:35:29,186.0,239.0,0.0,B02872,0.0,2018-01-01,2018-01-01 05:00:00,1514782800,...,sky is clear,01n,186,Manhattan,Penn Station/Madison Sq West,Yellow Zone,239,Manhattan,Upper West Side South,Yellow Zone
3,2018-01-01 04:31:00,2018-01-01 04:49:11,244.0,138.0,0.0,B02764,0.0,2018-01-01,2018-01-01 05:00:00,1514782800,...,sky is clear,01n,244,Manhattan,Washington Heights South,Boro Zone,138,Queens,LaGuardia Airport,Airports
4,2018-01-01 21:19:45,2018-01-01 21:45:29,66.0,162.0,0.0,B02765,0.0,2018-01-01,2018-01-01 21:00:00,1514840400,...,sky is clear,01d,66,Brooklyn,DUMBO/Vinegar Hill,Boro Zone,162,Manhattan,Midtown East,Yellow Zone
5,2018-01-01 13:03:52,2018-01-01 13:30:04,129.0,125.0,0.0,B02883,0.0,2018-01-01,2018-01-01 13:00:00,1514811600,...,sky is clear,01d,129,Queens,Jackson Heights,Boro Zone,125,Manhattan,Hudson Sq,Yellow Zone
6,2018-01-01 12:18:43,2018-01-01 12:38:30,13.0,189.0,0.0,B02869,0.0,2018-01-01,2018-01-01 12:00:00,1514808000,...,sky is clear,01n,13,Manhattan,Battery Park City,Yellow Zone,189,Brooklyn,Prospect Heights,Boro Zone
7,2018-01-01 00:20:22,2018-01-01 00:53:36,233.0,20.0,0.0,B02764,0.0,2018-01-01,2018-01-01 00:00:00,1514764800,...,sky is clear,01n,233,Manhattan,UN/Turtle Bay South,Yellow Zone,20,Bronx,Belmont,Boro Zone
8,2018-01-01 20:39:03,2018-01-01 20:48:02,26.0,26.0,1.0,B02869,0.0,2018-01-01,2018-01-01 21:00:00,1514840400,...,sky is clear,01d,26,Brooklyn,Borough Park,Boro Zone,26,Brooklyn,Borough Park,Boro Zone
9,2018-01-01 20:11:19,2018-01-01 20:30:15,157.0,229.0,0.0,B02510,0.0,2018-01-01,2018-01-01 20:00:00,1514836800,...,sky is clear,01d,157,Queens,Maspeth,Boro Zone,229,Manhattan,Sutton Place/Turtle Bay North,Yellow Zone
